<a href="https://colab.research.google.com/github/Sdebnath-astro/PRNSGeometry/blob/main/PRNSGeometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is a numerical Python numerical relativity code named **PRNSGeometry** that computes the different geometric components of the spacetime of a rotating proto-neutron star based on the paper: *Debnath & Majumder (2024); Dynamics of a Proto-Neutron Star Under Spacetime Rotation.*

In [ ]:
import sympy as sp
from sympy import symbols, diff, sin
from IPython.display import display, Math

# Define symbols with LaTeX formatting
G, M, c, rho, t00_LL, P, Q_mu_nu, J = symbols('G M c rho t00_LL P Q_mu_nu J')
t, r, theta, phi = symbols('t r \\theta \\phi')
x = [t, r, theta, phi]

# Define the line element components
g_tt = -(1 - 2 * G * M / (c**2 * r) + 8 * sp.pi * G / (c**2) * rho * (1 + 1/(c**2) * (t00_LL + 3 * P * G)) + 2 / (c**4 * r**3) * diff(diff(Q_mu_nu, t), t)) * c**2
g_rr = (1 - 2 * G * M / (c**2 * r))**(-1)
g_theta_theta = r**2
g_phi_phi = r**2 * sin(theta)**2
g_tp = -2 * G * J / (c**3 * r) * sin(theta)**2

# Define the metric tensor
g = [[g_tt, 0, 0, g_tp],
     [0, g_rr, 0, 0],
     [0, 0, g_theta_theta, 0],
     [g_tp, 0, 0, g_phi_phi]]

# Simplify the metric tensor components
g_mat = sp.Matrix(g).applyfunc(sp.simplify)

# Compute the inverse of the metric matrix
ginv_mat = g_mat.inv().applyfunc(sp.simplify)

# Function to calculate Christoffel symbols more efficiently
def christoffel_symbols(g, g_inv, x):
    Chr = [[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for l in range(4):
        for m in range(4):
            for n in range(4):
                term = 0
                for k in range(4):
                    term += g_inv[l, k] * (diff(g[k, m], x[n]) + diff(g[k, n], x[m]) - diff(g[m, n], x[k])) / 2
                Chr[l][m][n] = term
    return Chr

# Compute the Christoffel symbols
Chr = christoffel_symbols(g_mat, ginv_mat, x)

# Function to compute Riemann tensor more efficiently
def riemann_tensor(Chr, x):
    R = [[[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    R[i][j][k][l] = diff(Chr[i][j][l], x[k]) - diff(Chr[i][j][k], x[l])
                    for m in range(4):
                        R[i][j][k][l] += Chr[i][m][k] * Chr[m][j][l] - Chr[i][m][l] * Chr[m][j][k]
    return R

# Compute the Riemann tensor
Riem = riemann_tensor(Chr, x)

# Compute the Ricci tensor
Ric = [[sum(Riem[k][i][k][j] for k in range(4)).simplify() for j in range(4)] for i in range(4)]

# Compute the Ricci scalar
Rscal = sum(ginv_mat[i, j] * Ric[i][j] for i in range(4) for j in range(4)).simplify()

# Identity matrix
identity_matrix = sp.eye(4)

# Compute the maximal symmetric part of the Riemann tensor
max_sym = [[[[Riem[i][j][k][l] - Rscal / (4 * 3) * (
    identity_matrix[i, k] * g_mat[j, l] - identity_matrix[i, l] * g_mat[j, k]
) for l in range(4)] for k in range(4)] for j in range(4)] for i in range(4)]

# Convert tensors to LaTeX format
Chr_latex = [[[sp.latex(Chr[i][j][k]) for k in range(4)] for j in range(4)] for i in range(4)]
Riem_latex = [[[[sp.latex(Riem[i][j][k][l]) for l in range(4)] for k in range(4)] for j in range(4)] for i in range(4)]
Ric_latex = [[sp.latex(Ric[i][j]) for j in range(4)] for i in range(4)]
Rscal_latex = sp.latex(Rscal)
max_sym_latex = [[[[sp.latex(max_sym[i][j][k][l]) for l in range(4)] for k in range(4)] for j in range(4)] for i in range(4)]

# Display the results in LaTeX format
display(Math(r"\text{Metric tensor } \mathbf{g}:"))
display(Math(r"\mathbf{g} = " + sp.latex(g_mat)))
display(Math(r"\text{Inverse metric tensor } \mathbf{g}^{-1}:"))
display(Math(r"\mathbf{g}^{-1} = " + sp.latex(ginv_mat)))

display(Math(r"\text{Christoffel symbols } \Gamma:"))
for i in range(4):
    for j in range(4):
        for k in range(4):
            display(Math(r"\Gamma^{" + sp.latex(i) + r"}_{" + sp.latex(j) + sp.latex(k) + r"} = " + Chr_latex[i][j][k]))

display(Math(r"\text{Riemann curvature tensor } R:"))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                display(Math(r"R^{" + sp.latex(i) + r"}_{" + sp.latex(j) + sp.latex(k) + sp.latex(l) + r"} = " + Riem_latex[i][j][k][l]))

display(Math(r"\text{Ricci tensor } \mathbf{R}:"))
for i in range(4):
    for j in range(4):
        display(Math(r"R_{" + sp.latex(i) + sp.latex(j) + r"} = " + Ric_latex[i][j]))

display(Math(r"\text{Ricci scalar } R:"))
display(Math(r"R = " + Rscal_latex))

display(Math(r"\text{Maximal symmetric part of the Riemann tensor (max\_sym)}:"))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                display(Math(r"\text{max\_sym}^{" + sp.latex(i) + r"}_{" + sp.latex(j) + sp.latex(k) + sp.latex(l) + r"} = " + max_sym_latex[i][j][k][l]))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
!pip install IPython
from IPython.display import display, Math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [ ]:
import sympy as sp
from sympy import symbols, diff

# Define symbols
G, M, c, rho, t00_LL, P, Q_mu_nu, J = symbols('G M c rho t00_LL P Q_mu_nu J')
t, r, theta, phi = symbols('t r \\theta \\phi')
x = [t, r, theta, phi]

# Define the line element components
g_tt = -(1 - 2 * G * M / (c**2 * r) + 8 * sp.pi * G / (c**2) * rho * (1 + 1/(c**2) * (t00_LL + 3 * P * G)) + 2 / (c**4 * r**3) * diff(diff(Q_mu_nu, t), t)) * c**2
g_rr = (1 - 2 * G * M / (c**2 * r))**(-1)
g_theta_theta = r**2
g_phi_phi = r**2 * sp.sin(theta)**2
g_tp = -2 * G * J / (c**3 * r) * sp.sin(theta)**2

# Define the metric tensor
g = [[g_tt, 0, 0, g_tp],
     [0, g_rr, 0, 0],
     [0, 0, g_theta_theta, 0],
     [g_tp, 0, 0, g_phi_phi]]

# Simplify the metric tensor components
g_mat = sp.Matrix(g).applyfunc(sp.simplify)

# Compute the inverse of the metric matrix
ginv_mat = g_mat.inv().applyfunc(sp.simplify)

# Function to calculate Christoffel symbols more efficiently
def christoffel_symbols(g, g_inv, x):
    Chr = [[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for l in range(4):
        for m in range(4):
            for n in range(4):
                term = 0
                for k in range(4):
                    term += g_inv[l, k] * (diff(g[k, m], x[n]) + diff(g[k, n], x[m]) - diff(g[m, n], x[k])) / 2
                Chr[l][m][n] = term
    return Chr

# Compute the Christoffel symbols
Chr = christoffel_symbols(g_mat, ginv_mat, x)

# Function to compute Riemann tensor more efficiently
def riemann_tensor(Chr, x):
    R = [[[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    R[i][j][k][l] = diff(Chr[i][j][l], x[k]) - diff(Chr[i][j][k], x[l])
                    for m in range(4):
                        R[i][j][k][l] += Chr[i][m][k] * Chr[m][j][l] - Chr[i][m][l] * Chr[m][j][k]
    return R

# Compute the Riemann tensor
Riem = riemann_tensor(Chr, x)

# Compute the Ricci tensor
Ric = [[sum(Riem[k][i][k][j] for k in range(4)).simplify() for j in range(4)] for i in range(4)]

# Compute the Ricci scalar
Rscal = sum(ginv_mat[i, j] * Ric[i][j] for i in range(4) for j in range(4)).simplify()

# Define the Weyl curvature tensor
def weyl_tensor(Riem, Ric, Rscal, g_mat, ginv_mat):
    C = [[[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for rho in range(4):
        for sigma in range(4):
            for mu in range(4):
                for nu in range(4):
                    term1 = Riem[rho][sigma][mu][nu]
                    term2 = (1/2) * (
                        g_mat[rho, mu] * Ric[nu][sigma] - g_mat[rho, nu] * Ric[mu][sigma] +
                        g_mat[sigma, nu] * Ric[mu][rho] - g_mat[sigma, mu] * Ric[nu][rho]
                    )
                    term3 = (Rscal / 6) * (
                        g_mat[rho, mu] * g_mat[nu, sigma] - g_mat[rho, nu] * g_mat[mu, sigma]
                    )
                    C[rho][sigma][mu][nu] = (term1 - term2 + term3)
    return C

# Compute the Weyl curvature tensor
Weyl = weyl_tensor(Riem, Ric, Rscal, g_mat, ginv_mat)

# Convert the Weyl tensor to LaTeX format
Weyl_latex = [[[[sp.latex(Weyl[i][j][k][l]) for l in range(4)] for k in range(4)] for j in range(4)] for i in range(4)]

# Display the Weyl tensor in LaTeX format
display(Math(r"\text{Weyl curvature tensor } C:"))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                display(Math(r"C^{" + sp.latex(i) + r"}_{" + sp.latex(j) + sp.latex(k) + sp.latex(l) + r"} = " + Weyl_latex[i][j][k][l]))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
import sympy as sp
from sympy import symbols, diff, I

# Define symbols
G, M, c, rho, t00_LL, P, Q_mu_nu, J = symbols('G M c rho t00_LL P Q_mu_nu J')
t, r, theta, phi = symbols('t r \\theta \\phi')
x = [t, r, theta, phi]

# Define the line element components
g_tt = -(1 - 2 * G * M / (c**2 * r) + 8 * sp.pi * G / (c**2) * rho * (1 + 1/(c**2) * (t00_LL + 3 * P * G)) + 2 / (c**4 * r**3) * diff(diff(Q_mu_nu, t), t)) * c**2
g_rr = (1 - 2 * G * M / (c**2 * r))**(-1)
g_theta_theta = r**2
g_phi_phi = r**2 * sp.sin(theta)**2
g_tp = -2 * G * J / (c**3 * r) * sp.sin(theta)**2

# Define the metric tensor
g = [[g_tt, 0, 0, g_tp],
     [0, g_rr, 0, 0],
     [0, 0, g_theta_theta, 0],
     [g_tp, 0, 0, g_phi_phi]]

# Simplify the metric tensor components
g_mat = sp.Matrix(g).applyfunc(sp.simplify)

# Compute the inverse of the metric matrix
ginv_mat = g_mat.inv().applyfunc(sp.simplify)

# Function to calculate Christoffel symbols more efficiently
def christoffel_symbols(g, g_inv, x):
    Chr = [[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for l in range(4):
        for m in range(4):
            for n in range(4):
                term = 0
                for k in range(4):
                    term += g_inv[l, k] * (diff(g[k, m], x[n]) + diff(g[k, n], x[m]) - diff(g[m, n], x[k])) / 2
                Chr[l][m][n] = term
    return Chr

# Compute the Christoffel symbols
Chr = christoffel_symbols(g_mat, ginv_mat, x)

# Function to compute Riemann tensor more efficiently
def riemann_tensor(Chr, x):
    R = [[[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    R[i][j][k][l] = diff(Chr[i][j][l], x[k]) - diff(Chr[i][j][k], x[l])
                    for m in range(4):
                        R[i][j][k][l] += Chr[i][m][k] * Chr[m][j][l] - Chr[i][m][l] * Chr[m][j][k]
    return R

# Compute the Riemann tensor
Riem = riemann_tensor(Chr, x)

# Compute the Ricci tensor
Ric = [[sum(Riem[k][i][k][j] for k in range(4)).simplify() for j in range(4)] for i in range(4)]

# Compute the Ricci scalar
Rscal = sum(ginv_mat[i, j] * Ric[i][j] for i in range(4) for j in range(4)).simplify()

# Define the Weyl curvature tensor
def weyl_tensor(Riem, Ric, Rscal, g_mat, ginv_mat):
    C = [[[[0 for _ in range(4)] for _ in range(4)] for _ in range(4)] for _ in range(4)]
    for rho in range(4):
        for sigma in range(4):
            for mu in range(4):
                for nu in range(4):
                    term1 = Riem[rho][sigma][mu][nu]
                    term2 = (1/2) * (
                        g_mat[rho, mu] * Ric[nu][sigma] - g_mat[rho, nu] * Ric[mu][sigma] +
                        g_mat[sigma, nu] * Ric[mu][rho] - g_mat[sigma, mu] * Ric[nu][rho]
                    )
                    term3 = (Rscal / 6) * (
                        g_mat[rho, mu] * g_mat[nu, sigma] - g_mat[rho, nu] * g_mat[mu, sigma]
                    )
                    C[rho][sigma][mu][nu] = (term1 - term2 + term3)
    return C

# Compute the Weyl curvature tensor
Weyl = weyl_tensor(Riem, Ric, Rscal, g_mat, ginv_mat)

# Define the null tetrad vectors
l = sp.Matrix([[(r**2 + 0**2) / g_rr, 1, 0, 0]])
n = sp.Matrix([[1/2 * (r**2 + 0**2), -1/2 * g_rr, 0, 0]])
m = sp.Matrix([[0, 0, 1/sp.sqrt(2) * r, I/sp.sqrt(2) * r * sp.sin(theta)]])
m_bar = sp.Matrix([[0, 0, 1/sp.sqrt(2) * r, -I/sp.sqrt(2) * r * sp.sin(theta)]])

# Calculate the Weyl scalars
def weyl_scalar_component(C, l, n, m, m_bar, i, j, k, l_):
    return -C[i][j][k][l_]

Psi0 = weyl_scalar_component(Weyl, l, n, m, m_bar, 0, 1, 0, 1)
Psi1 = weyl_scalar_component(Weyl, l, n, m, m_bar, 0, 1, 0, 1)
Psi2 = weyl_scalar_component(Weyl, l, n, m, m_bar, 0, 2, 2, 1)
Psi3 = weyl_scalar_component(Weyl, l, n, m, m_bar, 1, 3, 3, 2)
Psi4 = weyl_scalar_component(Weyl, l, n, m, m_bar, 1, 3, 3, 2)

# Convert the Weyl scalars to LaTeX format
Psi0_latex = sp.latex(Psi0.simplify())
Psi1_latex = sp.latex(Psi1.simplify())
Psi2_latex = sp.latex(Psi2.simplify())
Psi3_latex = sp.latex(Psi3.simplify())
Psi4_latex = sp.latex(Psi4.simplify())

# Display the Weyl scalars in LaTeX format
display(Math(r"\text{Weyl scalar } \Psi_0: " + Psi0_latex))
display(Math(r"\text{Weyl scalar } \Psi_1: " + Psi1_latex))
display(Math(r"\text{Weyl scalar } \Psi_2: " + Psi2_latex))
display(Math(r"\text{Weyl scalar } \Psi_3: " + Psi3_latex))
display(Math(r"\text{Weyl scalar } \Psi_4: " + Psi4_latex))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>